In [ ]:
from nbtemplate import display_header
display_header('PiSox_trades.ipynb')

# PiSox trades

In this notebook, I study the PiSox PSF in more detail and describe some performance trades for PiSox.

In [ ]:
import copy
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
import marxs.visualization.mayavi
from marxs.source import PointSource, FixedPointing
from marxs.visualization.mayavi import plot_object, plot_rays
from marxs.simulator import KeepCol

import sys
sys.path.append('../')
from redsox.redsox import xyz2zxy
from redsox.pisox import PerfectPisox, conf


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
instrum = PerfectPisox()
keep_prob = KeepCol('probability')
instrum.postprocess_steps.append(keep_prob)

my_source = PointSource(coords=SkyCoord(30., 30., unit='deg'), energy=0.277 * u.keV,
                        polarization=120. * u.degree,
                        geomarea=instrum.elements[0].area)
my_pointing = FixedPointing(coords=SkyCoord(30., 30., unit='deg'),
                            reference_transform=xyz2zxy)

photons_in = my_source.generate_photons(1 * u.ks)
photons_in = my_pointing(photons_in)

photons = instrum(photons_in.copy())


## Check the PSF

Since the mirror taken in these simulations is a simplistic approximation for RedSox and the PiSox mirror differs from that it's not clear at all that the mirror does the right thing. However, for the things we want to check in this notebook, we really do not need the right PSF, just one that is approximately the right size.

In [ ]:
# using sin(x) = x
((77 * u.micron) / (conf['f'] * u.mm) * u.rad).to(u.arcsec)

In [ ]:
fig, axes = plt.subplots(ncols=2)
ind = (photons['order'] == 0) & (photons['probability'] > 0) #& (photons['order_L1'] == 0)
axes[0].scatter(photons['y'][ind], photons['z'][ind], alpha=.01)
ind = ind & (photons['order_L1'] == 0)
axes[1].scatter(photons['y'][ind], photons['z'][ind], alpha=.01)
fig.subplots_adjust(wspace=.4)
for ax in axes:
    ax.set_xlabel('y position [mm]')
    ax.set_ylabel('z position [mm]')

The figure above shows a scatter plot of the distribution of 0th order photons on the detector. Note the very different scales between the two axes for the left plot. The right plot shows a zoom into the central part of the PSF, which reveals the bow-tie shape that we wanted to set up for this simulation. Fundamentally, the root cause of the bow-tie shape is the fact that we sub-aperture the mirrors. For any mirror, the scatter (from figure errors and particualte contamination) in the plane of the reflection is generally larger than the scatter perpendicular to the plane of reflection. This spreads out the photons more in one direction than the other and we choose the sub-aperturing such that the narrow direction of the bow-tie (the $y$-axis in the PiSoX coordinate system) is along the dispersion direction.

The left graph shows the shape of the PSF on a larger scale (note that the $z$ range shown is much larger than the $y$ range). The dots "above" and "below" the main PSF (around $z=0$ mm) are caused by photons diffracted from the L1 support structure of the CAT gratings. The L1 support is oriented perpendicular to the grating bars and acts as a diffraction grating itself.

In [ ]:
print('standard dev in x: {:6.4f} mm and in y: {:6.4f} mm'.format(np.std(photons['det_x'][ind]),
                                                             np.std(photons['det_y'][ind])))

These numbers are very close to the expected mirror performance, which confirms that the scattering coefficients in the mirror model are chosen correctly.

## Radius of curvature for curved gratings

The grating efficiency of CAT gratings changes with the blaze angle. If the blaze angle is small, i.e. the rays hit the grating parallel to the surface normal, then many rays will pass "straight through" to the zeroth order. Higher blaze angles favour higher orders. PiSox is designed for the blaze angle of 0.8 degree to maximise the number of photons that are diffracted into the first order where they will hit the multi-layer mirror at the blaze peak. We position the gratings such that a ray hitting the grating center has a blaze angle of 0.8 degrees. Since the gratings are located in a converging beam the blaze angle for rays hitting a flat grating near the edges is different from the nominal blaze angle. Fewer photons are diffracted into the first order and the effective area is reduced.

This can be corrected by bending the gratings. If the grating surface follows a cylinder with a radius of curvature that matches the distance of the grating from the focal point, that effect can be almost entirely be compensated. The axis of that cylinder is parallel to the cross-dispersion direction. In other words, the grating is curved along the long side. On the other hand, bending almost every grating with a different radius of curvature increases the complexity and thus cost and schedule risk dramatically. Here, I study four different scenarios:

- Gratings are flat.
- All gratings are bend with the same radius which is chosen to be close to the average distance beween grating and focal point.
- We use two different radii of curvature for the upper and lower part of the grating staircase.
- Each grating is curved individually.

In [ ]:
confnobend = copy.deepcopy(conf)
# Set bend to False and perform the bending according to individual prescriptions below.
confnobend['bend'] = False

In [ ]:
instrumnobend = PerfectPisox(confnobend)

In [ ]:
from redsox.gratings import bend_gratings
from redsox.pisox import PiGrid

instrumbend2 = PerfectPisox(confnobend)
bend2 = []
for g in instrumbend2.elements_of_class(PiGrid):
    elempos = np.stack(g.elem_pos)
    r, gamma, beta = g.cart2sph(elempos[:, 0, 3], elempos[:, 1, 3], elempos[:, 2, 3])
    bend2.append(np.mean(r))
    bend_gratings(g.elements, r_elem=bend2[-1])

In [ ]:
from marxs.missions.mitsnl.catgrating import CATL1L2Stack
from marxs.math.utils import h2e

instrumbendall = PerfectPisox(confnobend)

for e in instrumbendall.elements_of_class(CATL1L2Stack):
    # Just need some cart2sph (it's a static method anyway), so just reuse "g" fro mthe cell above
    r, gamma, beta = g.cart2sph(*h2e(e.geometry['center']))
    bend_gratings([e], r_elem=r)

In [ ]:
photonsnobend = instrumnobend(photons_in.copy())
photonsbend2 = instrumbend2(photons_in.copy())
photonsbendall = instrumbendall(photons_in.copy())

In [ ]:
fig, ax = plt.subplots()
plist = [photonsnobend, photons, photonsbend2, photonsbendall]
plabels = ['flat gratings', f'radius {conf["bend"]} mm', f'radius {bend2[0]:5.0f} / {bend2[1]:5.0f} mm', 'individual radii']
for p, l in zip(plist, plabels):
    out = ax.hist(np.rad2deg(p['blaze']), bins=np.arange(0, 2., .03), alpha=.5, label=l)
ax.legend()
ax.set_xlabel('Blaze angle of photons [degree]')
out = ax.set_ylabel('Number of photons')

fig.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/curvature.pdf', bbox_inches='tight')

The plot shows a histogram of the blaze angles of photons for the different scenarios. The distribution is very wide for flat gratings, but significantly more peaked for bend gratings in any scenario.

In [ ]:
fig, ax = plt.subplots()
plist = [photonsnobend, photons, photonsbend2, photonsbendall]
plabels = ['flat gratings', f'radius {conf["bend"]} mm', f'{bend2[0]:5.0f} / {bend2[1]:5.0f} mm', 'individual radii']
for p, l in zip(plist, plabels):
    out = ax.plot(np.rad2deg(p['blaze']), p['facet'], '.', alpha=.3, label=l)
ax.legend()
ax.set_xlabel('Blaze angle of photons [degree]')
out = ax.set_ylabel('Number of photons')

This plot shows a different way of looking at the same data. Shown in a scatter plot of grating facet ID and observed blaze angle, where the facet ID is sorted by (but not proprotional to) the position along the optical axis. Again, for flat gratings, the blaze angle span a wide range. For bend gratings, the photons concentrate around 0.8 degrees. In the scenario with a single grating radius, the distribution is widest at the top and the bottom of the plot and narrower for gratings near the middle where the radius is closer to the average radius. In the scenario with two radii (one for upper and one for the lower grating sector) the distribution is narrowest in the middle of each sector, but even when the radius of curvature is chosen individually for each grating, the blaze angle is not exactly 0.8 degrees because the gratings also have a finite witdh in cross-dispersion direction.

In [ ]:
from astropy.table import Table
from redsox.analysis import fractional_aeff

aeff = []
for p, l in zip(plist, plabels):
    aeff.append(fractional_aeff(p) * instrum.elements[0].area.to(u.cm**2))
aeff = np.stack(aeff)
    
out = Table({'scenario': plabels, 'imaging': aeff[:, 0], 'polarimetry': aeff[:, 1]})
out['imaging'].format='{:6.2f}'
out['polarimetry'].format='{:6.3f}'
out

This table shows the effective areas on the imaging and polarimatery channel for a representative energy of 0.277 keV. Flat gratings have the best imaging performance, but severely reduce the effective area of the more important polarimetry channel. There is little performace difference between the different bending options studied here, so I conclude that the simplest option (all gratings have the same curvature) should be the baseline design for PiSox.

## How wide should the gratings be?

PiSox uses a basline design with CAT gratings of 30 mm * 10 mm, where the long edge is parallel to the dispersion direction. In this trade, I test the choice of 10 mm width for the corss-dispersion direction. CAT gratings can be manufactured in larger sizes, but the ideal surface on which the gratings need to be placed is saddle-shaped. On the other hand, the design of the gratings requires the grating normal to be roughly perpendicular to the incoming rays. If the angle between ray and grating becomes large, then the support structures that hold the grating bars in place, in particular to so-called L2 support structure that is 0.5 mm deep, would cast large shadows and thus reduce the effective area. The grating membranes are fixed to the metal grating holder, which in turn is mounted to a larger mechanical structure. All these block some fraction of the area and thus reduce the number of rays that make it through to the detector.

So, there are two competing effects: On the one hand, larger gratings reduce the number of and thus area lost to these mounting structures; on the other hand, we have the constrained that the CAT grating normals are close to the direction of the incoming rays. Thus, the larger the grating, the more do the regions on the edges deviate from the surface on which the diffraction should happen. Instead, some rays hit the grating where it is located "above" the theoretical surface, and some "below". In the the first case, photons are diffracted too far, on the latter to little. Both can cause photons to arrive at the ML mirror at a position that does not match the Bragg peak for their wavelength.

In this trade, I run simulations with different CAT grating sizes.

In [ ]:
conf_grat = copy.deepcopy(conf)

In [ ]:
photons = instrum(photons_in.copy())

In [ ]:
def compare_grating_size(width, photons, conf):
    conf_grat = copy.deepcopy(conf)
    conf_grat['gratingzoom'] = [0.25, 15., width/2]
    instrum = PerfectPisox(conf_grat)
    photons = instrum(photons.copy())
    ind = photons['CCD_ID'] == 1
    return photons['probability'][ind].sum()

In [ ]:
width = [2., 4., 5., 6., 7., 8., 9., 10., 11., 12., 14., 16., 20., 25., 30.]

In [ ]:
flux = [compare_grating_size(w, photons_in.copy(), conf) for w in width]

In [ ]:
narrow_frames = copy.deepcopy(conf)
narrow_frames['gratingframe'] = [0, .5, .5]
fluxnarrow = [compare_grating_size(w, photons_in.copy(), narrow_frames) for w in width]

In [ ]:
plt.plot(width, flux / np.max(flux), label='1.5 mm frames')
plt.plot(width, fluxnarrow / np.max(flux), label='0.5 mm frames')
plt.ylabel('Photon flux after LGML')
plt.xlabel('width of gratings [mm]')
plt.legend()
plt.savefig('/Users/hamogu/MITDropbox/my_poster/20_SPIE_Polarimetry/SPIE2020-GoPiSox/catdimensions.pdf', bbox_inches='tight')

For grating holders with wide frames the ideal width is between 5 and 10 mm. Since the total effective area plateaus in that range, we can see that 10 mm wide gratings are a good choice for the design baseline. Narrower gratings can achieve te same effective area, but at a larger cost and complexity. 